## Load Data

In [27]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms, datasets
from torch.utils.data import DataLoader
import torch.nn.functional as F

In [28]:
# Define data transformations
train_transforms = transforms.Compose([
    transforms.Resize((100, 100)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
])

val_transforms = transforms.Compose([
    transforms.Resize((100, 100)),
    transforms.ToTensor(),
])

# Load the dataset
train_dataset = datasets.ImageFolder(root='melanoma_cancer_dataset/train', transform=train_transforms)
val_dataset = datasets.ImageFolder(root='melanoma_cancer_dataset/test', transform=val_transforms)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [29]:
dataiter = iter(train_loader)
images, labels = next(dataiter)

# Check the size of the first image and its label in the batch
print(f"Size of image: {images[0].size()}")
print(f"Label: {labels[0]}")

Size of image: torch.Size([3, 100, 100])
Label: 0


## CNN Model

In [30]:
class CNN(nn.Module):
    def __init__(self, num_classes=2):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        # After three rounds of pooling, size is 100 / 2 / 2 / 2 = 12.5, which rounds down to 12
        # Conv3 output is 64 channels by 12x12 spatial size
        self.fc1 = nn.Linear(64 * 12 * 12, 512)
        self.fc2 = nn.Linear(512, num_classes)
        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        # Flatten the tensor for the fully connected layer
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [31]:
model = CNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [32]:
for epoch in range(5):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 20 == 19:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 20:.3f}')
            running_loss = 0.0

print('Finished Training')

[1,    20] loss: 0.596
[1,    40] loss: 0.443
[1,    60] loss: 0.427
[1,    80] loss: 0.388
[1,   100] loss: 0.389
[1,   120] loss: 0.314
[1,   140] loss: 0.364
[1,   160] loss: 0.366
[1,   180] loss: 0.355
[1,   200] loss: 0.346
[1,   220] loss: 0.288
[1,   240] loss: 0.296
[1,   260] loss: 0.323
[1,   280] loss: 0.314
[1,   300] loss: 0.276
[2,    20] loss: 0.323
[2,    40] loss: 0.325
[2,    60] loss: 0.289
[2,    80] loss: 0.297
[2,   100] loss: 0.316
[2,   120] loss: 0.347
[2,   140] loss: 0.285
[2,   160] loss: 0.281
[2,   180] loss: 0.263
[2,   200] loss: 0.298
[2,   220] loss: 0.297
[2,   240] loss: 0.276
[2,   260] loss: 0.291
[2,   280] loss: 0.356
[2,   300] loss: 0.275
[3,    20] loss: 0.276
[3,    40] loss: 0.287
[3,    60] loss: 0.272
[3,    80] loss: 0.269
[3,   100] loss: 0.259
[3,   120] loss: 0.270
[3,   140] loss: 0.307
[3,   160] loss: 0.265
[3,   180] loss: 0.291
[3,   200] loss: 0.260
[3,   220] loss: 0.251
[3,   240] loss: 0.234
[3,   260] loss: 0.223
[3,   280] 

In [33]:
PATH = './cnn.pth'
torch.save(model.state_dict(), PATH)

In [34]:
correct = 0
total = 0
with torch.no_grad():
    for data in val_loader:
        images, labels = data
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)  # Get the index of the max log-probability
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the model on the test images: {100 * correct / total}%')


Accuracy of the model on the test images: 88.7%
